In [1]:
import numpy as np

In [2]:
# Sample text
text = "Continuous Bag of Words (CBOW) is one of the architectures used in the Word2Vec framework for learning word embeddings. CBOW is designed to predict a target word based on its context which consists of surrounding words within a fixed window size".split()

# Create vocabulary
word_to_index = {word: i for i, word in enumerate(set(text))}
index_to_word = {i: word for word, i in word_to_index.items()}
VOCAB_SIZE = len(word_to_index)


In [3]:
# One-hot encoding

def one_hot_vector(word):
    vec = np.zeros(VOCAB_SIZE)
    vec[word_to_index[word]] = 1
    return vec

In [4]:
def generate_training_data(text, window_size=1):
    X_train, Y_train = [], []
    
    for i in range(1, len(text) - 1):  # Ignore first and last word
        context_left = one_hot_vector(text[i - 1])
        context_right = one_hot_vector(text[i + 1])
        target = one_hot_vector(text[i])
        
        # Concatenating both context words
        X_train.append(np.concatenate((context_left, context_right)))
        Y_train.append(target)
    
    return np.array(X_train), np.array(Y_train)


In [5]:
# Generate input and target data
X_train, Y_train = generate_training_data(text)
X_train.shape, Y_train.shape

((39, 70), (39, 35))

In [6]:
# Model parameters
INPUT_SIZE = 2 * VOCAB_SIZE  # Twice the vocabulary size due to concatenation
HIDDEN_SIZE = 10
OUTPUT_SIZE = VOCAB_SIZE

# Initialize weights and biases
W1 = np.random.randn(HIDDEN_SIZE, INPUT_SIZE) * 0.01
b1 = np.zeros((HIDDEN_SIZE, 1))
W2 = np.random.randn(OUTPUT_SIZE, HIDDEN_SIZE) * 0.01
b2 = np.zeros((OUTPUT_SIZE, 1))

# Activation functions
def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z))
    return exp_Z / exp_Z.sum(axis=0, keepdims=True)

def relu_derivative(Z):
    return Z > 0

# Forward propagation
def forward_propagation(X):
    Z1 = np.dot(W1, X.T) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

# Backward propagation
def backward_propagation(Z1, A1, Z2, A2, X, Y, alpha):
    global W1, b1, W2, b2
    
    m = X.shape[0]
    dZ2 = A2 - Y.T
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2) * relu_derivative(Z1)
    dW1 = (1 / m) * np.dot(dZ1, X)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2

In [7]:

# Training the model
EPOCHS = 1000
ALPHA = 0.1
track_word = "of"  # Word to track during training
track_index = word_to_index[track_word]

for epoch in range(EPOCHS):
    Z1, A1, Z2, A2 = forward_propagation(X_train)
    backward_propagation(Z1, A1, Z2, A2, X_train, Y_train, ALPHA)
    
    if epoch % 100 == 0:
        loss = -np.mean(np.sum(Y_train * np.log(A2.T + 1e-8), axis=1))
        embedding_vector = W1[:, track_index]  # Extracting embedding of tracked word
        print(f"Epoch {epoch}: Loss = {loss:.4f}")
        print(f"Embedding for '{track_word}': {embedding_vector}")


Epoch 0: Loss = 3.5554
Embedding for 'of': [-0.00017233  0.0080626  -0.00184576  0.01039047 -0.01084843 -0.01407626
  0.005025   -0.00321657 -0.00291391 -0.01705789]
Epoch 100: Loss = 3.5077
Embedding for 'of': [ 0.00128587  0.01443856 -0.00276269  0.01246338 -0.01130911 -0.01732401
  0.00624983 -0.00244083 -0.00473443 -0.01705789]
Epoch 200: Loss = 3.4821
Embedding for 'of': [ 0.00354933  0.02296072 -0.00234266  0.01693458 -0.01446552 -0.0228438
  0.00682977 -0.00173089 -0.00689776 -0.01693279]
Epoch 300: Loss = 3.4673
Embedding for 'of': [ 0.00732259  0.035925   -0.00139335  0.02509717 -0.01731113 -0.03012987
  0.00690382 -0.00366802 -0.01170799 -0.01849587]
Epoch 400: Loss = 3.4547
Embedding for 'of': [ 0.01384386  0.05656166  0.0004662   0.03952402 -0.02107966 -0.04629145
  0.00587515 -0.00856227 -0.01874481 -0.02511609]
Epoch 500: Loss = 3.4362
Embedding for 'of': [ 0.02494572  0.09018875  0.00385412  0.06473348 -0.02563999 -0.07250702
  0.00232601 -0.01824084 -0.02983973 -0.03536